
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.



## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [2]:
import ee
import geemap.foliumap as geemap
import webbrowser
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context
ee.Authenticate()
ee.Initialize(project='ee-juliaszymanska142')

aoi = ee.Geometry.Point([19.9449799, 50.0646501])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [3]:
image = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
         .filterBounds(aoi)
         .filterDate('2021-07-01', '2021-07-31')
         .sort('CLOUD_COVER')
         .first())




print('Image ID:', image.getInfo()['id'])


Image ID: LANDSAT/LC08/C02/T1/LC08_188025_20210728



## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [4]:
def get_radiance_band(band):
    M = ee.Number(image.get(f'RADIANCE_MULT_BAND_{band}'))
    A = ee.Number(image.get(f'RADIANCE_ADD_BAND_{band}'))
    DN = image.select(f'B{band}')
    return DN.multiply(M).add(A).rename(f'B{band}_rad')

radiance = ee.Image.cat([
    get_radiance_band(2),
    get_radiance_band(3),
    get_radiance_band(4)
])





## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [5]:
import math

ESUN = {
    'B2': 2067,
    'B3': 1893,
    'B4': 1603,
}

sun_elev_val = image.getInfo().get('properties', {}).get('SUN_ELEVATION')
if sun_elev_val is None:
    raise ValueError("Brakuje wartości 'SUN_ELEVATION' w metadanych obrazu.")
sun_elev = ee.Number(sun_elev_val)
sun_zenith_rad = ee.Number(90).subtract(sun_elev).multiply(math.pi).divide(180)
cos_theta = sun_zenith_rad.cos()

def to_toa(band):
    L = radiance.select(f'B{band}_rad')
    esun_bn = ee.Number(ESUN[f'B{band}'])
    return L.multiply(math.pi).divide(esun_bn.multiply(cos_theta)).rename(f'B{band}_toa')

toa_image = ee.Image.cat([
    to_toa(2),
    to_toa(3),
    to_toa(4)
])


## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [6]:
rgb_vis_dn = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 10000,
    'gamma': 1.4
}
rgb_vis_toa = {
    'bands': ['B4_toa', 'B3_toa', 'B2_toa'],
    'min': 0.0,
    'max': 0.4,
    'gamma': 1.2
}

Map = geemap.Map(center=[50.0647, 19.945], zoom=10)
Map.addLayer(image, rgb_vis_dn, 'Original DN RGB')
Map.addLayer(toa_image.clamp(0, 0.4), rgb_vis_toa, 'TOA Reflectance RGB')
Map.addLayer(aoi, {'color': 'red'}, 'AOI – Kraków')

Map.save("Landsat_2021_krakow_map.html")


webbrowser.open("Landsat_2021_krakow_map.html")

True


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
